### 임포트

In [1]:
import torch
import os
import sys
import numpy as np
import random

# custom
from util import *
from transformers import GPT2Tokenizer
from AAC_Prefix.AAC_Prefix import * # network
from Train import *

### 기타 값들 설정

In [2]:
# prefix vector 크기 설정
temporal_prefix_size = 15 # 0 or 15
global_prefix_size = 11 # 0 or 11

prefix_size = temporal_prefix_size + global_prefix_size 

# mapping network가 사용할 transformer의 스펙 설정
transformer_num_layers = {"temporal_num_layers" : 4, "global_num_layers" : 4}
prefix_size_dict = {"temporal_prefix_size" : temporal_prefix_size, "global_prefix_size" : global_prefix_size}


data_dir = './AudioCaps'
MODEL_NAME = 'add_exp_train_audiocaps_test_clotho'

epochs = 50
LR = 5e-5

TEST_BATCH_SIZE = 5
TRAIN_BATCH_SIZE = 75

random_seed=2766
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.benchmark=False
torch.backends.cudnn.deterministic=True
np.random.seed(random_seed)
random.seed(random_seed)  

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda' if USE_CUDA else 'cpu')

### Tokenizer, Dataloader 불러오기

In [3]:
tokenizer_type = 'GPT2'
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab_size = None

In [4]:
train_dataloader = CreateDataloader(tokenizer, data_dir, TRAIN_BATCH_SIZE, 'train', prefix_size, is_TrainDataset = True, tokenizer_type = tokenizer_type)
test_dataloader_clotho = CreateDataloader(tokenizer, './Clotho', TEST_BATCH_SIZE, 'evaluation', prefix_size, is_TrainDataset = False, tokenizer_type = tokenizer_type)

get dataset...: 100%|██████████████████████| 1045/1045 [00:06<00:00, 167.92it/s]


### 학습결과 정리하는 폴더 생성하기

In [5]:
directory = "./Train_record/params_" + MODEL_NAME
try:
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print("Error: Failed to create the directory.")


### 모델 초기화

In [6]:
model = get_AAC_Prefix(tokenizer, 
                        vocab_size = vocab_size, Dataset = 'AudioCaps',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = True, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

# 다른 데이터셋으로 평가하는데 이 때 조건이 Header는 freeze하는 조건이 들어있었다. 그래서 Header freeze 해줌
for param in model.language_header.parameters():
    param.requires_grad = False


/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


use GPT2 Tokenizer
temporal feature's mapping network : num_head = 8 num_layers = 4
global feature ver's mapping network : num_head = 8 num_layers = 4
Encoder freezing
GPT2 freezing


### 학습 & 평가

In [7]:
Train(model, LR, train_dataloader, test_dataloader_clotho,
    epochs, model_name = MODEL_NAME, beam_search = True, device = device,
    Dataset = 'AudioCaps', test_dataloader_other_dataset = None)

/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Eval using dataset...: 100%|████████████████| 1045/1045 [04:06<00:00,  4.24it/s]


loading annotations into memory...
0:00:00.007327
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 70696 tokens at 473341.46 tokens per second.
PTBTokenizer tokenized 10386 tokens at 120137.35 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 8176, 'reflen': 9775, 'guess': [8176, 7131, 6086, 5049], 'correct': [3504, 945, 280, 58]}
ratio: 0.8364194373400679
Bleu_1: 0.352
Bleu_2: 0.196
Bleu_3: 0.113
Bleu_4: 0.061
computing METEOR score...
METEOR: 0.109
computing Rouge score...
ROUGE_L: 0.277
computing CIDEr score...
CIDEr: 0.168
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.4 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

...

loading annotations into memory...
0:00:00.006988
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 70696 tokens at 496100.36 tokens per second.
PTBTokenizer tokenized 10007 tokens at 109871.30 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 7747, 'reflen': 9717, 'guess': [7747, 6702, 5657, 4622], 'correct': [3432, 924, 273, 58]}
ratio: 0.7972625295872391
Bleu_1: 0.344
Bleu_2: 0.192
Bleu_3: 0.111
Bleu_4: 0.060
computing METEOR score...
METEOR: 0.111
computing Rouge score...
ROUGE_L: 0.274
computing CIDEr score...
CIDEr: 0.181
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.4 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

SPICE evaluation took: 17.11 s
SPICE: 0.073
computing SPIDEr score...
SPIDEr: 0.127

Training time : 5:02:13
